In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/credit/application_record.csv')

record = pd.read_csv('data/credit/credit_record.csv')

In [3]:
print(f"Shape: {data.shape}")
print(data.columns)
print(f"Shape: {record.shape}")
print(record.columns)

Shape: (438557, 18)
Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'],
      dtype='object')
Shape: (1048575, 3)
Index(['ID', 'MONTHS_BALANCE', 'STATUS'], dtype='object')


Kaggle Notebook: https://www.kaggle.com/code/rikdifos/credit-card-approval-prediction-using-ml/notebook

## Feature Engineering

**Target variable**

- 0: 1-29 days past due 
- 1: 30-59 days past due 
- 2: 60-89 days overdue 
- 3: 90-119 days overdue 
- 4: 120-149 days overdue 
- 5: Overdue or bad debts, write-offs for more than 150 days 
- C: paid off that month 
- X: No loan for the month


TARGET VARIABLE: User at risk (1/0). 1 if overdue for more than 60 days (label 2,3,4,5)

In [4]:
record.STATUS.value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

In [5]:
# Create variable "begin_month" --> age of the account
begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
begin_month = begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 

new_data = pd.merge(data,begin_month,how="left",on="ID") #merge to record data

C:\Users\giorg\AppData\Local\Temp\ipykernel_54148\1215647021.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))


In [6]:
record['dep_value'] = np.where(record['STATUS'].isin(['2','3','4','5']), 'Yes', None)

# Flag customers who have been late 60+ days at least once
cpunt = record.groupby('ID').count()
cpunt['dep_value'] = np.where(cpunt['dep_value'] >0, 'Yes','No')
cpunt = cpunt[['dep_value']]

new_data = pd.merge(new_data, cpunt, how='inner', on='ID')

new_data['target'] = new_data['dep_value']
new_data['target'] = np.where(new_data['target']=='Yes', 1, 0)

new_data.target.value_counts()


target
0    35841
1      616
Name: count, dtype: int64

#### Binary Variables

**Gender**

In [7]:
print(new_data['CODE_GENDER'].value_counts())

new_data['CODE_GENDER'] = np.where(new_data['CODE_GENDER']=='M', 1, 0)

CODE_GENDER
F    24430
M    12027
Name: count, dtype: int64


In [8]:
new_data['CODE_GENDER'].isnull().sum()

0

**Own a car**

In [9]:
print(new_data['FLAG_OWN_CAR'].value_counts())

new_data['FLAG_OWN_CAR'] = np.where(new_data['FLAG_OWN_CAR']=='Y', 1, 0)

FLAG_OWN_CAR
N    22614
Y    13843
Name: count, dtype: int64


**Own Realty**

In [10]:
print(new_data['FLAG_OWN_REALTY'].value_counts())

new_data['FLAG_OWN_REALTY'] = np.where(new_data['FLAG_OWN_REALTY']=='Y', 1, 0)

FLAG_OWN_REALTY
Y    24506
N    11951
Name: count, dtype: int64


**Own Phone**

In [11]:
print(new_data['FLAG_PHONE'].value_counts())

FLAG_PHONE
0    25709
1    10748
Name: count, dtype: int64


**Own Work Phone**

In [12]:
new_data['FLAG_WORK_PHONE'] = np.where(new_data['FLAG_WORK_PHONE']=='Y', 1, 0)

**Own Email**

In [13]:
new_data['FLAG_EMAIL'].value_counts()

FLAG_EMAIL
0    33186
1     3271
Name: count, dtype: int64

#### Numerical Variables

**Children Count**

In [14]:
new_data['CNT_CHILDREN'].value_counts()

CNT_CHILDREN
0     25201
1      7492
2      3256
3       419
4        63
5        20
14        3
7         2
19        1
Name: count, dtype: int64

In [15]:
new_data['CNT_CHILDREN'] = np.where(new_data['CNT_CHILDREN']>=2, '2+ childrens',
                                    np.where(new_data['CNT_CHILDREN']==1, '1', '0'))

print(new_data['CNT_CHILDREN'].value_counts())
new_data = pd.get_dummies(new_data, columns=['CNT_CHILDREN'])

CNT_CHILDREN
0               25201
1                7492
2+ childrens     3764
Name: count, dtype: int64


**Income**

In [16]:
new_data['AMT_INCOME_TOTAL'].describe()

count    3.645700e+04
mean     1.866857e+05
std      1.017892e+05
min      2.700000e+04
25%      1.215000e+05
50%      1.575000e+05
75%      2.250000e+05
max      1.575000e+06
Name: AMT_INCOME_TOTAL, dtype: float64

In [17]:
new_data['AMT_INCOME_TOTAL'] = new_data['AMT_INCOME_TOTAL']/10000

In [18]:
new_data['AMT_INCOME_TOTAL'] = pd.qcut(new_data['AMT_INCOME_TOTAL'], q = 3, labels = ["low","medium", "high"])
new_data['AMT_INCOME_TOTAL'].value_counts()

AMT_INCOME_TOTAL
low       14473
high      11282
medium    10702
Name: count, dtype: int64

In [19]:
new_data = pd.get_dummies(new_data, columns=['AMT_INCOME_TOTAL'])

**Age**

In [20]:
new_data['AGE'] = new_data['DAYS_BIRTH']/-365

new_data['AGE'] = pd.qcut(new_data['AGE'], q = 3, labels = ["low","medium", "high"])

In [21]:
new_data['AGE'].value_counts()

AGE
low       12156
high      12152
medium    12149
Name: count, dtype: int64

In [22]:
new_data = pd.get_dummies(new_data, columns=['AGE'])

**Working Years**

In [23]:
new_data['YEARS_EMPLOYED'] = - new_data['DAYS_EMPLOYED'] / 365	
new_data['YEARS_EMPLOYED'] = np.where(new_data['YEARS_EMPLOYED']<0, np.nan,new_data['YEARS_EMPLOYED'])

new_data['YEARS_EMPLOYED'] = pd.qcut(new_data['YEARS_EMPLOYED'], q = 5, labels = ["lowest","low","medium","high","highest"])
new_data['YEARS_EMPLOYED'].value_counts()

YEARS_EMPLOYED
low        6083
lowest     6072
highest    6065
high       6063
medium     6039
Name: count, dtype: int64

In [24]:
new_data = pd.get_dummies(new_data, columns=['YEARS_EMPLOYED'])

**Family Size**

In [25]:
new_data['CNT_FAM_MEMBERS'].value_counts()

CNT_FAM_MEMBERS
2.0     19463
1.0      6987
3.0      6421
4.0      3106
5.0       397
6.0        58
7.0        19
15.0        3
9.0         2
20.0        1
Name: count, dtype: int64

In [26]:
new_data['CNT_FAM_MEMBERS'] = np.where(new_data['CNT_FAM_MEMBERS']>=3, '3+ members', 
                                       np.where(new_data['CNT_FAM_MEMBERS']==2, '2',
                                                np.where(new_data['CNT_FAM_MEMBERS']==1, '1', '0')))

In [27]:
new_data = pd.get_dummies(new_data, columns=['CNT_FAM_MEMBERS'])

#### Categorical Variables

**Occupation**

In [28]:
# Define the occupation categories
laborwk_categories = ['Cleaning staff', 'Cooking staff', 'Drivers', 'Laborers', 'Low-skill Laborers', 'Security staff', 'Waiters/barmen staff']
officewk_categories = ['Accountants', 'Core staff', 'HR staff', 'Medicine staff', 'Private service staff', 'Realty agents', 'Sales staff', 'Secretaries']
hightecwk_categories = ['Managers', 'High skill tech staff', 'IT staff']

new_data['OCCUPATION_TYPE'] = np.where(new_data['OCCUPATION_TYPE'].isin(laborwk_categories), 'labor', 
                                       np.where(new_data['OCCUPATION_TYPE'].isin(officewk_categories), 'office',
                                                np.where(new_data['OCCUPATION_TYPE'].isin(hightecwk_categories), 'hightec', 'other')))

In [29]:
new_data.OCCUPATION_TYPE.value_counts()

OCCUPATION_TYPE
other      11323
labor      10496
office     10183
hightec     4455
Name: count, dtype: int64

In [30]:
new_data = pd.get_dummies(new_data, columns=['OCCUPATION_TYPE'])

**Income**

In [31]:
new_data['NAME_INCOME_TYPE'] = np.where(new_data['NAME_INCOME_TYPE'].isin(['Pensioner', 'Student']), 'State servant' ,new_data['NAME_INCOME_TYPE'])

In [32]:
new_data.NAME_INCOME_TYPE.value_counts()

NAME_INCOME_TYPE
Working                 18819
State servant            9148
Commercial associate     8490
Name: count, dtype: int64

In [33]:
## Create dummy variables - one hot encoding
new_data = pd.get_dummies(new_data, columns=['NAME_INCOME_TYPE'])

**House Type**

In [34]:
new_data['NAME_HOUSING_TYPE'].value_counts()

NAME_HOUSING_TYPE
House / apartment      32548
With parents            1776
Municipal apartment     1128
Rented apartment         575
Office apartment         262
Co-op apartment          168
Name: count, dtype: int64

In [35]:
new_data = pd.get_dummies(new_data, columns=['NAME_HOUSING_TYPE'])

**Acedemic level**

In [36]:
new_data['NAME_EDUCATION_TYPE'].value_counts()

NAME_EDUCATION_TYPE
Secondary / secondary special    24777
Higher education                  9864
Incomplete higher                 1410
Lower secondary                    374
Academic degree                     32
Name: count, dtype: int64

In [37]:
new_data['NAME_EDUCATION_TYPE'] = np.where(new_data['NAME_EDUCATION_TYPE']=='Academic degree', 'Higher education', new_data['NAME_EDUCATION_TYPE'])

# dummies
new_data = pd.get_dummies(new_data, columns=['NAME_EDUCATION_TYPE'])

**Marriage Condition**

In [38]:
new_data['NAME_FAMILY_STATUS'].value_counts()

NAME_FAMILY_STATUS
Married                 25048
Single / not married     4829
Civil marriage           2945
Separated                2103
Widow                    1532
Name: count, dtype: int64

In [39]:
new_data = pd.get_dummies(new_data, columns=['NAME_FAMILY_STATUS'])

## Models

In [40]:
new_data.columns

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'begin_month', 'dep_value', 'target', 'CNT_CHILDREN_0',
       'CNT_CHILDREN_1', 'CNT_CHILDREN_2+ childrens', 'AMT_INCOME_TOTAL_low',
       'AMT_INCOME_TOTAL_medium', 'AMT_INCOME_TOTAL_high', 'AGE_low',
       'AGE_medium', 'AGE_high', 'YEARS_EMPLOYED_lowest', 'YEARS_EMPLOYED_low',
       'YEARS_EMPLOYED_medium', 'YEARS_EMPLOYED_high',
       'YEARS_EMPLOYED_highest', 'CNT_FAM_MEMBERS_1', 'CNT_FAM_MEMBERS_2',
       'CNT_FAM_MEMBERS_3+ members', 'OCCUPATION_TYPE_hightec',
       'OCCUPATION_TYPE_labor', 'OCCUPATION_TYPE_office',
       'OCCUPATION_TYPE_other', 'NAME_INCOME_TYPE_Commercial associate',
       'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Working',
       'NAME_HOUSING_TYPE_Co-op apartment',
       'NAME_HOUSING_TYPE_House / apartment',
       'NAME_HOUSING_TYPE_Municipal apartment',
       'NAME_HO

In [41]:
X = new_data.drop(['ID','target','dep_value'], axis=1)
y = new_data['target']

In [44]:
from sklearn.model_selection import train_test_split

ImportError: DLL load failed while importing _argkmin: The filename or extension is too long.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    stratify=y, test_size=0.3,
                                                    random_state = 10086)

**Logistic Regression**

In [45]:
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import seaborn as sns


model = LogisticRegression(C=0.8,
                           random_state=0,
                           solver='lbfgs')
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

print('Accuracy Score is {:.5}'.format(accuracy_score(y_test, y_predict)))
print(pd.DataFrame(confusion_matrix(y_test,y_predict)))

sns.set_style('white') 
class_names = ['0','1']
plot_confusion_matrix(confusion_matrix(y_test,y_predict),
                      classes= class_names, normalize = True, 
                      title='Normalized Confusion Matrix: Logistic Regression')

ImportError: DLL load failed while importing _argkmin: The filename or extension is too long.

In [46]:
import sklearn.metrics as metrics

ImportError: DLL load failed while importing _argkmin: The filename or extension is too long.